# Auditory Demonstrations 

These demos are wrappers around selected demos from the AUDITORY DEMONSTRATIONS CD
made in 1987 at the Institute for Perception Research (IPO)
Eindhoven, The Netherlands with
support by the Acoustical Society of America.

**Reference**:  [**Auditory Demonstrations**](https://research.tue.nl/en/publications/auditory-demonstrations), *A.J.M. Houtsma, Th.D. Rossing, W.M. Wagemakers*, Technische Universiteit Eindhoven, Institute for Perception Research, 1987.    More detailed documentation may be found  [here].(https://pure.tue.nl/ws/portalfiles/portal/79033010/402660.pdf)

The **Jupyter notebook** embedding allows for additional functionality such as viewing waveforms, spectrograms and feature functions in a simple framework.  You are encouraged to develop and testt your personal feature functiona and test them in view of the presented demonstrations. 

*Dirk Van Compernolle, June 2021*

In [ ]:
%matplotlib inline

from urllib.request import urlopen
from IPython.display import display, Audio, HTML, clear_output
import ipywidgets as widgets
import math
import numpy as np


### REQUIRED PACKAGES
# 
# You need the pyspch package to be installed to run these demos
# installing pyspch will automatically install additional packages such as soundfile, librosa
# 
try:
  import google.colab
  IN_COLAB = True 

except:
  IN_COLAB = False

try:
  import pyspch
except:
  ! pip install git+https://github.com/compi1234/pyspch.git
    
from pyspch import spectrogram as specg
from pyspch import audio
import pyspch.display as spch_disp

In [ ]:
# Energy function in dB
# we measure intensity as  10. * log10(SUM(x^2)/N) + 14
# - the +15 offset is arbitrary
# - the EPS of 1.e-10 yields a range of around 60dB
# 
EPS = 1.e-10
OFFSET = 15.0

def en_db(y,sample_rate=16000,f_shift=0.01):
    n_shift = int(f_shift*sample_rate)
    nfr = int(len(y)/n_shift)
    energy = np.zeros((nfr,))
    for ifr in range(0,nfr):
        energy[ifr] = np.sum(np.square(y[ifr*n_shift:(ifr*n_shift+n_shift)]))/float(n_shift)
    ftr = 10*np.log10(energy+EPS) + OFFSET
    return(ftr)


def ftr_en(fig,data):
    spch_disp.add_line_plot(fig,en_db(data),row=3,ylabel='Energy (dB)',yrange=[-50,0],dx=.01)

# a default boxed layout
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
demos = {}
demos['Demo4: The Decibel Scale (a)'] = {'filename':'TrackNo08.wav','split':12.,
                      'commentary':'Broadband Noise is reduced in 10 steps of 6 decibels. Demonstrations are repeated once.'}
demos['Demo4: The Decibel Scale (b)'] = {'filename':'TrackNo09.wav','split':7.,
                      'commentary':'Broadband noise is reduced in 15 steps of 3 decibels.'}
demos['Demo4: The Decibel Scale (c)'] = {'filename':'TrackNo10.wav','split':1.,
                      'commentary':'Broadband noise is reduced in 20 steps of 1 decibel.'}
demos['Demo4: The Decibel Scale (d)'] = {'filename':'TrackNo11.wav','split':7.,
                      'commentary':'Free-field speech of constant power at various distances from the microphone.'}

demos['Demo29: Effect of Tone Envelope on Timbre (a)'] = {'filename':'TrackNo54.wav','split':10.,
         'commentary': "You will hear a recording of a Bach chorale played on a piano"}
demos['Demo29: Effect of Tone Envelope on Timbre (b)'] = {'filename':'TrackNo55.wav','split':4.0,
         'commentary': "Now the same chorale will be played backwards"}
demos['Demo29: Effect of Tone Envelope on Timbre (c)'] = {'filename':'TrackNo56.wav','split':9.0,
         'commentary': "Finally the tape of the last recording is played backwards so that the chorale is heard forward again,but with an interesting difference"}




class AuditoryDemos(widgets.HBox):
    def __init__(self,feature=None):
        super().__init__()
            
        self.name = 'Demo4: The Decibel Scale (a)'
        self.demo = demos[self.name]
        self.root = 'http://homes.esat.kuleuven.be/~compi/demos/AuditoryDemonstrations/'
        self.fig = None
        self.dpi = 200
        self.feature = feature
        
        # create the widgets
        self.wg_name = widgets.Dropdown(options=demos.keys(),value=self.name,description="",layout=widgets.Layout(height='60px',width='100%'))
        self.wg_name.observe(self.update_demo,'value')
        self.output = widgets.Output()

        self.text_commentary = widgets.Output() 
        self.audio_commentary = widgets.Output()
        self.audio_demo = widgets.Output()
        self.left = widgets.VBox([self.output, self.audio_demo],layout=box_layout())
        self.left.layout.width = '60%'
        self.right = widgets.VBox([self.wg_name, self.text_commentary], layout=box_layout())
        self.right.layout.width = '40%'
        self.children = [self.left, self.right]
        self.update()
    
    def update_demo(self,change):
        self.name = change.new
        self.demo = demos[self.name]
        self.update()
        
    def update(self):
        with self.text_commentary:
            print("\n\nloading %s  ..." % self.name)
        
        self.wavdata, self.sample_rate = audio.load(self.root+self.demo['filename'],sample_rate=16000,mono=True)
            
        indx = int(self.demo['split']*self.sample_rate)
        instr_data = self.wavdata[:indx]
        demo_data = self.wavdata[indx:] 
        with self.text_commentary:
            clear_output(wait=True)
            print("\n",self.name,"\n")
            print(self.demo['commentary'])
        #with self.audio_commentary:
        #    clear_output(wait=True)
        #    print("Listen to the commentary")
        #    display(Audio(data=instr_data,rate=self.sample_rate,autoplay=False))
        with self.audio_demo:
            clear_output(wait=True)
            print("Listen to the demo")
            display(Audio(data=demo_data,rate=self.sample_rate,autoplay=False))            
            
        spg = specg.spectrogram(demo_data,sample_rate=self.sample_rate)
        if self.feature is None:
            self.fig = spch_disp.plot_spg(spg,wav=demo_data,sample_rate=self.sample_rate,dpi=self.dpi)
        else :
            self.fig = spch_disp.plot_spg(spg,wav=demo_data,sample_rate=self.sample_rate,dpi=self.dpi,figsize=(10,5),ftr_axis=True,ftr_height=2)
            self.feature(self.fig,demo_data)
        with self.output:
            clear_output(wait=True)
            display(self.fig)
           
        


## The Decibel Scale (Demo 4, Tracks 08-11) 

You will hear
- Broadband noise reduced in 10 steps of 6 decibels." 
- Broadband noise reduced in 15 steps of 3 decibels." 
- Broadband noise reduced in 20 steps of 1 decibels." 
- FreeField Speech at different distances from the Microphone."  

The figure contains following displays:
- the time waveform
- spectrogram
- feature fuction: energy on a dB scale

In [ ]:
AuditoryDemos(feature=ftr_en)

## The Effect of Tone Envelope on Timbre (Demo 29, Tracks 54-56)

- "You will hear a recording of a Bach chorale played on a piano" (Track No54)
- "Now the same chorale will be played backwards" (Track No55)
- "Finally the tape of the last recording is played backwards so that the chorale is heard forward again, but with an interesting difference". (Track No 56)

In [ ]:
AuditoryDemos(feature=ftr_en)